In [1632]:
import pandas as pd
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

%load_ext autoreload
%autoreload 2

from utils.utils import (
    generate_data,
    print_project_summary,
    student_assignment,
    process_applications
)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1633]:
file_location = '../data/Winter 25 Matching.xlsx'
deprioritized_students = ["hanyunl@uchicago.edu"]
application_df, forced_assignments = process_applications(file_location, deprioritized_students)

In [1634]:
technical_project_list = [
    'Morningstar', 'Fermi - GNN', 'Fermi - Simulations', 'Center for Living Systems',
    'Building Decarbonization Coalition', 'Argonne', 'CMAP', 'Invenergy'
]

data_to_process_dict = generate_data(application_df, technical_project_list)

Dropping 0 students because they do not have a computer and not required
Total Students available for matching: 111


In [1635]:
priority_weights = [10, 15, 25, 30]
max_students = {'Fermi - GNN': 3, 'RAFI - Grocery': 5}
all_project_list = data_to_process_dict.pop('all_project_list')
assignment_df = student_assignment(priority_weights = priority_weights, **data_to_process_dict,
                                   max_students_dict=max_students, preassigned_students=forced_assignments,
                                   number_of_projects_to_run=None, drop_projects=["University of Northern Iowa"], verbose=False)
print(f"\nAverage Ranking: {assignment_df.Ranking.mean():.2f}")
print(assignment_df.Ranking.value_counts())
print_project_summary(assignment_df, all_project_list)

assignment_df.to_csv('../data/test_1.csv', index=False)


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/hannifan/projects/the-clinic/.venv/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/sr/lrm99gp93rx47xspcx2z49gr0000gp/T/a9b5f6876fce48b8967bbbd7c46ae2ef-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/sr/lrm99gp93rx47xspcx2z49gr0000gp/T/a9b5f6876fce48b8967bbbd7c46ae2ef-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 1140 COLUMNS
At line 11655 RHS
At line 12791 BOUNDS
At line 14084 ENDATA
Problem MODEL has 1135 rows, 1292 columns and 6655 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 680 - 0.00 seconds
Cgl0002I 1012 variables fixed
Cgl0003I 10 fixed, 0 tightened bounds, 5 strengthened rows, 0 substitutions
Cgl0004I processed model has 84 rows, 253 columns (253 integer (252 of which binary)) and 506 elements
Cutoff increment incre

,Project Name,Number,High Priority,High-Med Priority,Med Priority,Low Priority,Experienced,Rankings,Average Ranking
0,RAFI - Grocery,5,5,0,0,0,2,"0,0,0,2,3",1.00
1,Argonne,4,4,0,0,0,1,"0,0,0,0",0.00
2,International Rescue Committee,4,4,0,0,0,1,"0,0,0,1",0.25
3,Food System 6,4,4,0,0,0,1,"0,0,0,1",0.25
4,CMAP,4,3,1,0,0,1,"0,0,1,1",0.50
5,Internet Equity Initiative,4,4,0,0,0,0,"0,0,1,2",0.75
6,IDI - Grievances,4,4,0,0,0,1,"0,0,1,2",0.75
7,UChicago Library,4,3,0,1,0,2,"0,0,0,3",0.75
8,Morningstar,4,4,0,0,0,1,"0,0,1,3",1.00
9,Invenergy,4,4,0,0,0,1,"0,1,1,2",1.00


In [1636]:
project_to_comp = 'CMAP'

list_of_not_assigned = list(assignment_df.loc[ (assignment_df.Priority == 1) & (assignment_df.Ranking.isna()), 'Email Address'])
prefs_df = data_to_process_dict['ranking'].copy()
prefs_df.loc[~(prefs_df.Ranking == 100.0), :].copy()

unmatched_prefs = prefs_df.loc[(prefs_df.loc[:, 'Project Name'] == project_to_comp) & prefs_df.loc[:, 'Email Address'].isin(list_of_not_assigned)
             & (prefs_df.Ranking < 100), :]

unmatched_info = pd.merge(unmatched_prefs, assignment_df.loc[:, ['Email Address', 'Experienced', 'Priority']], how='left', on='Email Address')

matched = assignment_df.loc[(assignment_df.loc[:, 'Project Assigned'] == project_to_comp), :]
display(matched.sort_values('Ranking'))

display(unmatched_info)

,Email Address,Priority,Experienced,Project Assigned,Ranking
10,annapoon@uchicago.edu,1,False,CMAP,0
102,xiaoyue1@uchicago.edu,2,True,CMAP,0
1,acherman@uchicago.edu,1,False,CMAP,1.0
49,jmdeleon@uchicago.edu,1,False,CMAP,1.0


,Email Address,Project Name,Ranking,Experienced,Priority


In [1637]:
# Priority students that were not assigned to a project
assignment_df[(assignment_df['Priority'] == 1) & (assignment_df['Project Assigned'].isna())]

,Email Address,Priority,Experienced,Project Assigned,Ranking


In [1638]:
# Non-priority students that were assigned to a project
assignment_df[(assignment_df['Project Assigned'].notna()) & (assignment_df['Priority'] > 1)]

,Email Address,Priority,Experienced,Project Assigned,Ranking
91,shaileshbolduc@uchicago.edu,3,True,UChicago Library,0
102,xiaoyue1@uchicago.edu,2,True,CMAP,0
